In [2]:
import os
import sys
import math
from ipywidgets import FloatProgress
from IPython.display import display
from numba import jit
import numpy as np #alloys numpy arrays to be used
import matplotlib as mpl #see next comment
import matplotlib.pyplot as plt #allows use of plt. to plot figures
import pandas as pd
import networkx as nx #https://networkx.readthedocs.io/en/stable/index.html  #Aric A. Hagberg, Daniel A. Schult and Pieter J. Swart, “Exploring network structure, dynamics, and function using NetworkX”, in Proceedings of the 7th Python in Science Conference (SciPy2008), Gäel Varoquaux, Travis Vaught, and Jarrod Millman (Eds), (Pasadena, CA USA), pp. 11–15, Aug 2008
from time import time
sys.path.append("/home/pg1g15/Documents/SEM_Container")
import SEM
from NXFunc import * #imports all my functions from MergerTreeGraphs 
h = SEM.Cosmo.Returnh()

Resolution cut, Scale factor cut, file headers, and input data 

In [3]:
Resolution = np.log(h) + 12
SF = 0.143
Header = ["scale", "id", "desc_scale", "desc_id", "num_prog", "pid", "upid", "desc_pid", "phantom", "sam_mvir", "mvir", "rvir", "rs", "vrms", "mmp", "scale_of_last_MM", "vmax", "x", "y", "z", "vx", "vy", "vz", "Jx", "Jy", "Jz", "Spin", "Breadth_first_ID", "Depth_first_ID", "Tree_root_ID", "Orig_halo_ID", "Snap_num", "Next_coprogenitor_depthfirst_ID", "Last_progenitor_depthfirst_ID", "Last_mainleaf_depthfirst_ID", "Tidal_Force", "Tidal_ID", "Rs_Klypin", "Mvir_all", "M200b", "M200c", "M500c", "M2500c", "Xoff", "Voff", "Spin_Bullock", "b_to_a", "c_to_a", "Ax", "Ay", "Az", "b_to_a500c", "c_to_a500c", "Ax500c", "Ay500c", "Az500c", "T/|U|", "M_pe_Behroozi", "M_pe_Diemer" ]
BolshoiFiles = os.listdir("/data/B-P50Mpc/Original_Data/")
print(BolshoiFiles)
Destination = "/home/pg1g15/Documents/SEM_Container/Merger_Trees/Test/"
BadTrees = []

['tree_1_0_1.dat', 'tree_0_1_1.dat', 'tree_1_0_0.dat', 'tree_1_1_1.dat', 'tree_0_1_0.dat', 'tree_0_0_1.dat', 'tree_0_0_0.dat', 'tree_1_1_0.dat']


1:Takes the raw bolshoi boxes.

2:Creates individual tree files.

3:Perfoms a mass cut. 

4:Numbers the files into size order.

In [ ]:
#processes boxs into individual trees spltting them into files names by their tree ID given by bolshoi 
for Box in BolshoiFiles:
    outfile = open("/data/B-P50Mpc/Header.dat", 'w')
    with open("/data/B-P50Mpc/Original_Data/%s" %(Box)) as infile:
        for line in infile:
            linestr = str.split(line)
            if linestr[0] == '#tree':
                outfile.close()
                Treenum = linestr[1]
                outfile = open("/data/B-P50Mpc/Processed/Tree%s.dat" %(Treenum), 'w')
            else:
                outfile.write(str(line))

In [ ]:
#Gets the files that were just split up                
TreeFiles = os.listdir("/data/B-P50Mpc/Processed/")

#does a resolution cut on the z=0 Halos
RemoveList = []
KeepList = []
for Tree in TreeFiles:
    with open("/data/B-P50Mpc/Processed/%s" %(Tree), "r") as File:
        for line in File:
            line = str.split(line)
            Test = line
            if(float(line[40]) > 10.0**Resolution):
                KeepList.append([Tree, float(line[40])])
                break
            else:
                RemoveList.append([Tree, float(line[40])])
                break

#prints the length of the list we intend to keep
print(len(KeepList))

#orders the files and numbers them by size (smallest first)
SortedKeepList = sorted(KeepList, key=lambda x : x[1])
for i, File in enumerate(SortedKeepList):
    os.system("cp /data/B-P50Mpc/Processed/%s /data/B-P50Mpc/Above12/Tree%s.dat" %(File[0], i) )

In [4]:
Files = os.listdir("/data/B-P50Mpc/Above12")
Files.remove('RedshiftCut')
Files.remove('.ipynb_checkpoints')
for File in Files:
    Files.append(File)
    df = pd.read_csv("/data/B-P50Mpc/Above12/%s" %(File), delim_whitespace = True, names = Header)
    df.M200c = np.log10(df.M200c) #turns m200c into a log10 m200c
    MPH = df.id[0] #identifys the id of the MP
    #Resolution Cut the DF
    DF = df[df.scale > SF]
    #Create network from DF  add more node attributes here for more detailed tree information
    MergerTree = nx.from_pandas_dataframe(DF.drop(0), "id", "desc_id", edge_attr = "mmp", create_using = nx.Graph())
    nx.set_node_attributes(MergerTree, 'scale', pd.Series(DF.scale.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'M200c', pd.Series(DF.M200c.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'mmp', pd.Series(DF.mmp.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'upid', pd.Series(DF.upid.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'pid', pd.Series(DF.pid.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'rvir', pd.Series(DF.rvir.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'rs', pd.Series(DF.rs.values, index=DF.id).to_dict())
    nx.set_node_attributes(MergerTree, 'vmax', pd.Series(DF.vmax.values, index=DF.id).to_dict())
    
    #gets the Main prog branch
    MPB = DefineMPB(MergerTree, MPH, MPB = [MPH])
    if 0.2 < MergerTree.node[MPB[-1]]['scale']:
        BadTrees.append(int(File[4:-4]))
        print(File)
        print(len(MPB))
        print(MergerTree.node[MPB[-1]]['scale'])
        continue
    
    #defines subhalos
    DefineSubHalos(MergerTree, MPH)
    
    #Removes the doubble count
    #RemoveDCM(MergerTree, MPH, MPB, 0, Doubble_Count= [])
    
    #Gets infall halos
    TreeList = MakeSEMTree(MergerTree, MPH, MPB, 0, MaxDepth = len(MPB))
    
    #Creates the output file
    Outfile = open("/data/B-P50Mpc/Above12/RedshiftCut/SEMunbinned/%s" %(File), "w")
    Outfile.write("START\n")
    
    #writes to the output file
    for Epoch in TreeList:
        MainProg = True
        for i in Epoch:
            if MainProg:
                #print(i)
                Outfile.write("mpro " + str(MergerTree.node[i]['scale']) +" "+ str(MergerTree.node[i]['M200c']) +" "+ str(MergerTree.node[i]['rvir']) +" "+ str(MergerTree.node[i]['rs']) +" "+ str(MergerTree.node[i]['vmax']) + "\n")
                MainProg = False
            else:
                Outfile.write( "sat " + str(MergerTree.node[i]['scale']) +" "+ str(MergerTree.node[i]['M200c']) +" "+ str(MergerTree.node[i]['rvir']) +" "+ str(MergerTree.node[i]['rs']) +" "+ str(MergerTree.node[i]['vmax']) + "\n")

/home/pg1g15/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in log10


Tree436.dat
135
0.21762
Tree7996.dat
135
0.21762
Tree794.dat
137
0.2075
Tree5005.dat
110
0.34418
Tree1084.dat
108
0.35431
Tree2126.dat
129
0.248
Tree580.dat
132
0.23281
Tree5036.dat
135
0.21762
Tree7184.dat
138
0.20244
Tree1006.dat
133
0.22775
Tree4780.dat
135
0.21762
Tree2323.dat
131
0.23787
Tree8258.dat
137
0.2075
Tree3933.dat
131
0.23787
Tree3005.dat
131
0.23787
Tree1912.dat
135
0.21762
Tree4485.dat
137
0.2075
Tree1869.dat
113
0.329
Tree1948.dat
135
0.21762
Tree2265.dat
122
0.28343
Tree1375.dat
134
0.22268
Tree5900.dat
130
0.24294
Tree4958.dat
138
0.20244
Tree4471.dat
138
0.20244
Tree458.dat
135
0.21762
Tree46.dat
121
0.2885
Tree8124.dat
132
0.23281
Tree5204.dat
137
0.2075
Tree6736.dat
116
0.31381
Tree7958.dat
137
0.2075
Tree5945.dat
122
0.28343
Tree5983.dat
108
0.35431
Tree2171.dat
131
0.23787
Tree3277.dat
138
0.20244
Tree6498.dat
137
0.2075
Tree1181.dat
136
0.21256
Tree3872.dat
131
0.23787
Tree1087.dat
138
0.20244
Tree6338.dat
136
0.21256
Tree8601.dat
113
0.329
Tree8609.dat
121
0.

KeyboardInterrupt: 

In [8]:
len(BadTrees)

168

In [7]:
SaveFile = Destination
StartTime = time()
TimeStep = 0
RedshiftStep = 0
MassAtLastWrite = 0
ResolvedMass = -300
Resolution = 8 #logmstar
Switch = True
Exception = False
TreeFiles = os.listdir("/data/B-P50Mpc/Above12/RedshiftCut/SEMunbinned/")
f = FloatProgress(min = 0, max = len(TreeFiles))
display(f)
for File in TreeFiles:
    Switch = True
    RedshiftStep = 0
    MassAtLastWrite = 0
    ResolvedMass = -300
    f.value += 1
    with open("/data/B-P50Mpc/Above12/RedshiftCut/SEMunbinned/%s" %(File), "r") as InputFile, open((SaveFile + "%s" %(File)), "w") as OutputFile:
        for LineStr in InputFile:
            Line = str.split(LineStr) 
            if Line[0] == "START":
                OutputFile.write("START\n")
            else:
                Line[1] = float(Line[1])
                Line[1] = (1 / Line[1]) - 1
                L1asT = SEM.RedshiftToTime(Line[1])
                Line[2] = float(Line[2])

                if Line[0] == "mpro":
                    if Switch == True:
                        if Line[2] == -np.inf:
                            print("-inf", File)
                            BadTrees.append(int(File[4:-4]))
                        OutputFile.write( "mpro  %s  %s  %s  %s\n" %(Line[1], Line[2], Line[3], Line[4]) )
                        MassAtLastWrite = Line[2]
                        Switch = False
                    elif L1asT >= (TimeStep + 0.1):
                        if 10**MassAtLastWrite - 10**Line[2] - 10**ResolvedMass > 0:                            
                            OutputFile.write( "1  acc  %s  %s  %s  1\n" %(Line[1], math.log10(10**MassAtLastWrite - 10**Line[2] - 10**ResolvedMass), Line[5]) ) 
                        else:
                            OutputFile.write( "1  acc  %s  %s  %s  -1\n" %(Line[1], math.log10(abs(10**MassAtLastWrite - 10**Line[2] - 10**ResolvedMass)), Line[5]) ) 
                        MassAtLastWrite = Line[2]
                        ResolvedMass = 0
                        try:
                            if Line[2] == -np.inf:
                                print("-inf", File)
                                BadTrees.append(int(File[4:-4]))
                            OutputFile.write( "mpro  %s  %s  %s  %s  %s\n" %(Line[1], Line[2], Line[3], Line[4], Line[5]) )
                        except:
                            print(File)
                            BadTrees.append(int(File[4:-4]))
                            Exception = True
                            break
                        RedshiftStep += 0.01
                elif Line[0] == "sat" and Line[2] > Resolution:
                    ResolvedMass = math.log10(10**ResolvedMass + 10**Line[2])
                    OutputFile.write( "1  %s  %s  %s  %s  %s\n" %(Line[1], Line[2], Line[3], Line[4], Line[5]) )
    if(Exception == True):
        Exception = False
        continue
print(BadTrees)
np.savetxt(SaveFile + "BadTrees.txt", np.array(list(set(BadTrees))).astype(int))

The installed widget Javascript is the wrong version. It must satisfy the semver range ~2.1.4.


-inf Tree531.dat
-inf Tree1683.dat
-inf Tree4198.dat
-inf Tree350.dat
-inf Tree3956.dat
-inf Tree702.dat
-inf Tree3302.dat
-inf Tree56.dat
-inf Tree565.dat
-inf Tree1887.dat
-inf Tree3444.dat
-inf Tree4814.dat
-inf Tree806.dat
-inf Tree5872.dat
-inf Tree2527.dat
-inf Tree1512.dat
-inf Tree1011.dat
-inf Tree1943.dat
-inf Tree2422.dat
-inf Tree5143.dat
-inf Tree6743.dat
-inf Tree1460.dat
-inf Tree670.dat
-inf Tree5609.dat
-inf Tree7022.dat
-inf Tree785.dat
-inf Tree347.dat
-inf Tree1858.dat
-inf Tree907.dat
-inf Tree1540.dat
-inf Tree889.dat
[436, 7996, 794, 5005, 1084, 2126, 580, 5036, 7184, 1006, 4780, 2323, 8258, 3933, 3005, 1912, 4485, 1869, 1948, 2265, 1375, 5900, 4958, 4471, 458, 46, 8124, 5204, 6736, 7958, 5945, 5983, 2171, 3277, 6498, 1181, 3872, 1087, 6338, 8601, 8609, 3886, 4410, 1223, 2417, 595, 1878, 2073, 7, 277, 6418, 1573, 1247, 677, 5947, 3279, 7287, 1487, 1961, 4411, 2898, 834, 203, 499, 7677, 6372, 3567, 6678, 1744, 299, 7008, 2759, 1994, 9082, 2086, 3183, 3533, 2565, 4

In [ ]:
plt.hist((list(set(BadTrees))))
plt.show()

In [ ]:
print(len(list(set(BadTrees))))
np.savetxt(SaveFile + "BadTrees.txt", np.array(list(set(BadTrees))).astype(int))

In [ ]:
%matplotlib inline
Redshift = (1/SF) -1
#Sets Attributes of root
nx.set_node_attributes(MergerTree, 'color', {MPH: 'blue'})

#makes color mapping for plotting hte nodes
nodes = MergerTree.nodes()
colors = [MergerTree.node[n]['color'] for n in nodes]

#makes the graph area
fig = plt.figure()
fig.set_axes
#plt.ylim(Redshift,0)
#plt.xlim(1, -3)
ax = fig.add_axes()

#Makes the positioning of the nodes
pos = hierarchy_pos(MergerTree, MPH, height = 3) #2nd input is the root (MP at z = 0)
#pos = MakePosition(MergerTree, MPH, MPB, MPH, 0)
#'draws' the graph
nx.draw_networkx(MergerTree, pos = pos, nodelist = nodes, node_color = colors, with_labels = False, node_size = 10, ax = ax)

In [ ]:
A = np.loadtxt(SaveFile + "BadTrees.txt")
B = np.append(A, 347)
np.savetxt(SaveFile + "BadTrees.txt", B.astype(int))